In [ ]:

import csv
import os
import sys
from docplex.mp.progress import *
from docplex.mp.progress import SolutionRecorder
import docplex.mp.model as cpx
import networkx as nx
import time
from config import get_config
from absl import flags
FLAGS = flags.FLAGS

In [1]:
class Solver:
    def __init__(self):
        pass
    def get_product(self,path):
        result = 1
        for edge in path:
            result = w_vars[l]*result
        return result
    def CPLEX_maximizing_QNU(self,wk_idx,network):
      
        opt_model = cpx.Model(name="QNUM")
        r_vars  = {(f,p): opt_model.continuous_var(lb=0, ub= network.max_edge_capacity,
                                  name="r_{0}_{1}".format(f,p))  for f in network.each_wk_flow_ids[wk_idx]
                                  for p in network.each_wk_flow_id_paths[wk_idx][f]}

        w_vars[l]  = {(l): opt_model.continuous_var(lb=0, ub= 1,
                                  name="w_{0}".format(l))
                                  for l in network.set_E}
        
        #Edge constraint
        for edge in network.set_E:
            opt_model.add_constraint(
                opt_model.sum(r_vars[f,p]
                for f in network.each_wk_flow_ids[wk_idx]
                for p in network.each_wk_flow_id_paths[wk_idx][f]
                if network.check_path_include_edge(edge,p))
                 <= network.d[edge] *(1- w_vars[edge]), ctname="edge_capacity_{0}".format(edge))
                
        objective = opt_model.sum(log(r_vars[f,p]*(3*self.get_product(p)-1)) 
                            for f in network.each_wk_flow_ids[wk_idx]
                            for p in network.each_wk_flow_id_paths[wk_idx][f]
                            for l in network.set_E
                              )


        # for maximization
        opt_model.maximize(objective)

    #     opt_model.solve()
        #opt_model.print_information()
        #try:
        opt_model.solve()


#         print('docplex.mp.solution',opt_model.solution)
        objective_value = -1
        try:
            if opt_model.solution:
                objective_value =opt_model.solution.get_objective_value()
        except ValueError:
            print(ValueError)
        return objective_value
    
    